In [36]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
from torch import tensor
import torch.nn.functional as F
from torchvision.utils import save_image
from torchvision import transforms, utils, datasets
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from PIL import Image
import datetime
from dateutil import parser
import csv

In [37]:
csvfile = os.path.join('../tkim6/riri145/post-data.csv')
df = pd.read_csv(csvfile, skiprows=21404, 
                 usecols=[0, 1, 2, 3, 4, 5, 6, 7],
                names=['id', 'date',  'caption', 'image_name', 'tagged_users', 'likes', 'comments', 'ad'])
def cleanup(caption):
    return caption.replace('#ad', '').replace('#sponsored', '').replace('#Ad', '').replace('#advertisement', '')
df['caption'] = df['caption'].apply(cleanup)

In [38]:
X = df.caption
Y = df.ad
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1, 1)
max_words = 1000
max_len = 150
sequences_matrix = np.load('sequences_matrix.npy')

In [39]:
if False:
    tok = Tokenizer(num_words=max_words)
    tok.fit_on_texts(X)
    sequences = tok.texts_to_sequences(X)
    sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)
    np.save('sequences_matrix.npy', sequences_matrix)
else:
    print(sequences_matrix.shape)

(199409, 150)


In [40]:
class Net(nn.Module):

    def __init__(self):
        """Load the pretrained ResNet-50 and replace top fc layer."""
        super(Net, self).__init__()
        resnet = models.resnet50(pretrained=True)
        modules = list(resnet.children())[:-2]  # delete the last fc layer.
        self.resnet = nn.Sequential(*modules)
        self.avgpool = nn.AvgPool2d(7)
#         self.fc1 = nn.Linear(2048, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        features = self.resnet(x)
        features = self.avgpool(features)
        features = features.view(features.size(0), -1)  # reshape
        #return features
#         features = F.relu(self.fc1(features))
#         features = F.relu(self.fc2(features))
#         features = self.fc3(features)
#         features = torch.sigmoid(features)
        return features

In [41]:
def RNN1():
    model = nn.Sequential(
        nn.Embedding(max_words, 64),
        nn.LSTM(64, max_words))
    return model

def RNN2():
    model = nn.Sequential(
        nn.ReLU(inplace=False),
        nn.Dropout(0.5),
#           nn.Dense(1),
        nn.Linear(150000, 2048),
#         nn.Linear(2000, 100),
#         nn.Linear(100,2),
        #nn.MaxPool1d(2, 1000),
        #nn.MaxPool2d(1, 75),
        nn.Sigmoid()
    )
        
    return model
        
def LinearGravityBong():
    model = nn.Sequential(
        nn.Linear(4096, 256),
        nn.Linear(256, 16),
        nn.Linear(16, 2),
        nn.Sigmoid()
    )
    return model


In [42]:
DEFAULT_DATA_PATH = '/home/tkim6/riri145/img/'
DEFAULT_SAVED_LABELS = '/home/tkim6/riri145/preloaded.pt'

class InstagramDataset(Dataset):
    '''
    Characterizes a dataset for PyTorch.
    '''
    def __init__(self, dataset_path=DEFAULT_DATA_PATH, 
                label_path = DEFAULT_SAVED_LABELS, transform=None):

        # Checks if pre-saved training labels are available.
        # If not, loads from csv file and saves to a .pt file
        # (pytorch default save extension) to be loaded up in the future.
        if not os.path.exists(label_path):


            # Creates dictionary to save all image names and labels
            
            def oneHot(label):
                if label == 1:
                    return torch.tensor([0, 1], dtype=torch.float32)
                else:
                    return torch.tensor([1, 0], dtype=torch.float32)
            new_labels = [oneHot(label) for label in df['labels'].values.tolist()]
            
            data_dict = {
                'image_names': df['image_names'].values.tolist(),
                'caption': df['caption'].values.tolist(),
                'labels': new_labels,
            }
            
            

            # Iterates through csv file and grabs image names + labels
            '''for idx, line in enumerate(csv_reader):
                if idx > 21404 and ('jpg' in line[3] or 'png' in line[3]):
                    data_dict['image_names'].append(line[3])
                    data_dict['caption'].append(line[2])
                    if int(line[-1]) == 1:
                        data_dict['labels'].append(torch.tensor([1, 0], dtype=torch.float32))
                    else:
                        data_dict['labels'].append(torch.tensor([0, 1], dtype=torch.float32))'''

            # Saves for easy loading next time
            if not os.path.isdir(label_path[:label_path.rfind('/')]):
                os.makedirs(label_path[:label_path.rfind('/')])
            torch.save(data_dict, label_path)

        # Otherwise, just load the pre-saved dict.
        else:
            data_dict = torch.load(label_path)

        # Saves state variables
        self.data_dict = data_dict
        self.dataset_path = dataset_path
        self.label_path = label_path
        self.transform = transform


    def __len__(self):
        '''Denotes the total number of samples'''
        return len(self.data_dict['labels'])


    def __getitem__(self, index):
        '''Generates one sample of data'''
        # Select sample
        image_name = self.data_dict['image_names'][index]
        
        X = torch.zeros(3, 224, 224)
        try:
            X = self.transform(Image.open(self.dataset_path + image_name))
        except:
            X = torch.zeros(3, 224, 224)
        y = self.data_dict['labels'][index]

        return X, np.asarray(sequences_matrix[index], dtype=int), y


def get_dataloaders(dataset_path=DEFAULT_DATA_PATH, 
                    label_path = DEFAULT_SAVED_LABELS, val_split=0.2, batch_sz=1,
                    num_threads=1, shuffle_val=True):
    '''
    Grabs dataloaders for train/val sets.
    
    Keyword arguments:
    > dataset_path (string) -- Path to folder where all dataset images are stored.
    > label_path (string) -- Path to saved labels (should be .pt file).
    > val_split (float) -- Fraction of training data to be used as validation set.
    > batch_sz (int) -- Batch size to be grabbed from DataLoader.
    > num_threads (int) -- Number of threads with which to load data.
    > shuffle_val (bool) -- Whether to shuffle validation set indices.

    Return value: (train_dataloader, test_dataloader)
    > train_dataloader -- a torch.utils.data.DataLoader wrapper around
        the specified dataset's training set.
    > val_dataloader -- a torch.utils.data.DataLoader wrapper around
        the specified dataset's validation set.
    '''

    # Describes the transforms we want. Using randomCrop and toTensor.
    transform = transforms.Compose([
            transforms.Resize((224, 224)), # 128 x 128 random crop of image.
            transforms.ToTensor(),
        ])

    # Constructs InstagramDataset to load data from
    dataset = InstagramDataset(dataset_path=dataset_path,
                                label_path=label_path, transform=transform)

    # Grabs train/val split
    num_train = df.shape[0]
    indices = list(range(num_train))
    split = int(np.floor(val_split * num_train))

    # Shuffle indices if ncessary for slicing val set
    if shuffle_val:
        np.random.shuffle(indices)

    # Performs train/val split
    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(valid_idx)

    # Constructs dataloader wrappers around InstagramDataset training and test sets
    train_dataloader = DataLoader(dataset, batch_size=batch_sz, 
                                  num_workers=num_threads, sampler=train_sampler)
    val_dataloader = DataLoader(dataset, batch_size=batch_sz, 
                                num_workers=num_threads, sampler=val_sampler)

    return (train_dataloader, val_dataloader)

In [43]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()

        _, targets = target.topk(maxk, 1, True, True)
        correct = pred.eq(targets.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res, pred.numpy(), targets.t().numpy()

In [44]:
def train(network1, network2, network69, linear_gravity_bong, train_loader, val_loader):
    # THIS IS BAD. PLS REFACTOR
    #print(network.forward)
    learning_rate = 1e-2
    momentum = 0.9
    optimizer = torch.optim.SGD(network1.parameters(), lr=learning_rate, momentum=momentum)
    criterion = nn.BCELoss()
    num_epochs = 10

    for epoch in range(num_epochs):
        batch_time = AverageMeter('Time', ':6.3f')
        data_time = AverageMeter('Data', ':6.3f')
        losses = AverageMeter('Loss', ':.4e')
        top1 = AverageMeter('Acc@1', ':6.2f')
        progress = ProgressMeter(len(train_loader), batch_time, data_time, losses, top1,
                                prefix="Epoch: [{}]".format(epoch))

        running_loss = 0.0
        end = time.time()
        for i, data in enumerate(train_loader):
            #print(i, data)
            input, caption, target = data
            # measure data loading time
            data_time.update(time.time() - end)

            '''# No GPU yet
            args = get_args()
            if args.gpu is not None:
                input = input.cuda(args.gpu, non_blocking=True)
            target = target.cuda(args.gpu, non_blocking=True)'''
            device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
            input.to(device)
            caption.to(device)
            target.to(device)

            output, (h, c) = network1(caption)
            output = output.reshape(1, max_len * max_words)
            output = network2(output)
            ensemble = torch.cat((output, network69(input)), dim=1)
            
            output = LinearGravityBong()(ensemble)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1 = accuracy(output, target)[0][0]
            losses.update(loss.item(), caption.size(0))
            top1.update(acc1[0], caption.size(0))
            #losses.update(loss.item(), input.size(0))
            #top1.update(acc1[0], input.size(0))

            # compute gradient and do SGD step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 20 == 0:
                progress.print(i)

            # Save weights
            if i % 100 == 99:
                print('outputs:', output)
                # validate(val_loader, network, criterion)
                if not os.path.isdir('weights/'):
                    os.makedirs('weights/')
                torch.save({
                        'network1': network1,
                        'network2': network2,
                        'network69': network69,
                        'linear_gravity_bong': linear_gravity_bong,
                        'optimizer': optimizer,
                        'criterion': criterion,
                    },
                    'weights/weights_epoch_' + str(epoch) + '_iteration_' + str(i).zfill(6) + '.pt')

In [45]:
def load_state_dict():
    '''
    Always loads the latest saved network state.
    '''
    all_weight_files = glob.glob('weights/*')
    all_weight_files = sorted(all_weight_files)
    print('Loading ' + all_weight_files[-1] + '...')
    state_dict = torch.load(all_weight_files[-1])
    print('Finished loading!')
    return state_dict

In [46]:
def validate(val_loader, model1, model2, model69, linear_model, criterion, args=None):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(len(val_loader), batch_time, losses, top1,
                             prefix='Test: ')

    print('\n --- BEGIN VALIDATION PASS --- ')

    # switch to evaluate mode
    model1.eval()
    model2.eval()
    model69.eval()
    linear_model.eval()

    guess = np.array([])
    actual = np.array([])
    
    with torch.no_grad():
        end = time.time()
        for i, (input, caption, target) in enumerate(val_loader):

            # No GPU at the moment
            # if args.gpu is not None:
            #     input = input.cuda(args.gpu, non_blocking=True)
            # target = target.cuda(args.gpu, non_blocking=True)

            # compute output
            device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
            input.to(device)
            caption.to(device)
            target.to(device)

            output, (h, c) = model1(caption)
            output = output.reshape(1, max_len * max_words)
            output = model2(output)
            ensemble = torch.cat((output, model69(input)), dim=1)
            
            output = linear_model(ensemble)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc_tuple = accuracy(output, target)
            acc1 = acc_tuple[0][0]
            #import pdb; pdb.set_trace()
            for val in acc_tuple[1][0]:
                assert(val==0 or val==1)
            for val in acc_tuple[2][0]:
                assert(val==0 or val==1)
            guess = np.concatenate((guess, acc_tuple[1][0]))
            actual = np.concatenate((actual, acc_tuple[2][0]))
            losses.update(loss.item(), input.size(0))
            top1.update(acc1[0], input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 20 == 0:
                progress.print(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f}'
              .format(top1=top1))

        print(' --- END VALIDATION PASS --- \n')
    print(actual)
    print(guess)
    print(metrics.classification_report(actual, guess, digits=3))
    return top1.avg

In [47]:
#model = MockupModel()
#model = RNN()
model1 = RNN1()
model2 = RNN2()
model69 = Net()
linear_gravity_bong = LinearGravityBong()
train_loader, val_loader = get_dataloaders()
#model = train(model1, model2, model69, linear_gravity_bong, train_loader, val_loader)
state_dict = load_state_dict()
validate(val_loader, state_dict['network1'], state_dict['network2'], state_dict['network69'],
    state_dict['linear_gravity_bong'], state_dict['criterion'])

Loading weights/weights_epoch_0_iteration_000499.pt...
Finished loading!

 --- BEGIN VALIDATION PASS --- 
Test: [    0/39925]	Time 11.948 (11.948)	Loss 7.1733e-01 (7.1733e-01)	Acc@1   0.00 (  0.00)
Test: [   20/39925]	Time  0.375 ( 1.033)	Loss 6.2119e-01 (7.0170e-01)	Acc@1 100.00 ( 38.10)
Test: [   40/39925]	Time  0.381 ( 0.717)	Loss 6.6480e-01 (7.0245e-01)	Acc@1 100.00 ( 39.02)
Test: [   60/39925]	Time  0.526 ( 0.615)	Loss 6.7165e-01 (7.0287e-01)	Acc@1 100.00 ( 39.34)
Test: [   80/39925]	Time  0.370 ( 0.575)	Loss 7.3103e-01 (7.0364e-01)	Acc@1   0.00 ( 39.51)
Test: [  100/39925]	Time  0.375 ( 0.536)	Loss 7.1850e-01 (7.0255e-01)	Acc@1   0.00 ( 40.59)
Test: [  120/39925]	Time  0.372 ( 0.510)	Loss 6.8483e-01 (7.0284e-01)	Acc@1 100.00 ( 41.32)
Test: [  140/39925]	Time  0.387 ( 0.492)	Loss 6.4956e-01 (7.0294e-01)	Acc@1 100.00 ( 40.43)


KeyboardInterrupt: 

In [ ]:
test_sequences = tok.texts_to_sequences(Y_test['caption'])
test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen=max_len)
pred = model.predict(test_sequences_matrix)